In [93]:
%gui osx
import py5
import numpy as np
import random

In [94]:
# Given three collinear points p, q, r, the function checks if 
# point q lies on line segment 'pr' 
def onSegment(p, q, r):
    if ( (q.x <= max(p.x, r.x)) and (q.x >= min(p.x, r.x)) and 
           (q.y <= max(p.y, r.y)) and (q.y >= min(p.y, r.y))):
        return True
    return False
  
def orientation(p, q, r):
    # to find the orientation of an ordered triplet (p,q,r)
    # function returns the following values:
    # 0 : Collinear points
    # 1 : Clockwise points
    # 2 : Counterclockwise
      
    # See https://www.geeksforgeeks.org/orientation-3-ordered-points/amp/ 
    # for details of below formula. 
      
    val = (float(q.y - p.y) * (r.x - q.x)) - (float(q.x - p.x) * (r.y - q.y))
    if (val > 0):
          
        # Clockwise orientation
        return 1
    elif (val < 0):
          
        # Counterclockwise orientation
        return 2
    else:
          
        # Collinear orientation
        return 0
  
# The main function that returns true if 
# the line segment 'p1q1' and 'p2q2' intersect.
def doIntersect(p1,q1,p2,q2):
    

    if ((p1==p2) or (p1==q2) or (q1==p2) or (q1==q2)):
        return False 
    # Find the 4 orientations required for 
    # the general and special cases
    o1 = orientation(p1, q1, p2)
    o2 = orientation(p1, q1, q2)
    o3 = orientation(p2, q2, p1)
    o4 = orientation(p2, q2, q1)
  
    # General case
    if ((o1 != o2) and (o3 != o4)):
        return True
    
    # If none of the cases
    return False
   

In [95]:
class mPoint:
    def __init__(self,inx, iny, index=-1, szomszedok={}):
        self.x =inx
        self.y =iny
        self.index=index
        self.szomszedok=szomszedok

        
    def __add__(self, other):
        return mPoint(self.x+other.x,self.y+other.y)
    
    def __sub__(self, other):
        return mPoint(self.x-other.x,self.y-other.y)
    
    def __str__(self):
        return str('('+str(self.index)+','+str(self.x)+','+str(self.y)+','+str(self.szomszedok)+')')
    
    def mag(self):
        return np.sqrt(self.magSq())
  
    def magSq(self):
        return self.x*self.x+self.y*self.y;
    
    def display(self):
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
    
    def display_elestol(self):
        py5.stroke_weight(20)
        py5.point(self.x, self.y)
        py5.stroke_weight(4)
        for csucs in self.szomszedok:
            py5.line(self.x,self.y,graf_csucsai.plist[csucs].x,graf_csucsai.plist[csucs].y)


class edge:
    global graf_elei
    def __init__(self, inStart,inEnd):
        self.s =inStart
        self.t =inEnd
        self.metszo=False
       
    def display(self):
        py5.stroke_weight(4)
        if self.metszo==True:
            py5.stroke(204, 102, 0)
        else:
            py5.stroke(153)
            
        py5.line(self.s.x,self.s.y,self.t.x,self.t.y)
    def metsz_e_egy_konkretat(self,masikel):
    
        p1=self.s
        q1=self.t
        p2=masikel.s
        q2=masikel.t
        if ((p1==p2) or (p1==q2) or (q1==p2) or (q1==q2)):
            return False 
        
        # Find the 4 orientations required for 
        # the general and special cases
        o1 = orientation(p1, q1, p2)
        o2 = orientation(p1, q1, q2)
        o3 = orientation(p2, q2, p1)
        o4 = orientation(p2, q2, q1)
  
        # General case
        if ((o1 != o2) and (o3 != o4)):
            return True
    
        # If none of the cases
        return False
   
        
        
    def metsz_e_valamit(self):
        ideig=False
        for el in graf_elei.plist:
            if self.metsz_e_egy_konkretat(el):
                ideig=True
        self.metszo=ideig
        return ideig
   
class edge_set:
    def __init__(self, plist=[]):
        self.plist=plist
        
    def display(self):
        for x in self.plist:
            x.display()
    def van_e_metszes(self):
        ideig=False
        for x in self.plist:
            if x.metsz_e_valamit():
                ideig=True
        return ideig
        
class dinamic_pointset:
    def __init__(self, plist=[]):
        self.plist=plist
        self.active=False
        self.act_point=mPoint(0,0,0,[])
        self.log=False         
       
    def display(self):
        for x in self.plist:
            x.display()
    
    def select(self):
        if self.log:
            print('select')
        
        ref_point=mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)
        ref_point.display()
        dist_bound=30
        min_dist=dist_bound
        min_point=mPoint(0,0)
        for p in self.plist:
            if dist(ref_point,p)<min_dist:
                min_dist=dist(ref_point,p)
                min_point=p
        
        if min_dist<dist_bound:
            if self.log:
                print('succes select')
        
            self.active=True
            self.act_point=min_point
        else:
            if self.log:
                print('not succes select')
     #   self.plist.append(ref_point)
        
    def update(self):
        #print('update')
        if self.active:
            self.act_point.x=py5.mouse_x-py5.width/2
            self.act_point.y=py5.mouse_y-py5.height/2
    
    def relase(self):
        if self.log:
            print('re')
        
        self.active=False
   
    def do_something(self):
        if self.active:
            self.relase()
        else:
            self.select()
    
    def add_point(self,p):
        if not self.active:
            self.plist.append(p)
        else:
            if self.log:
                print('cant add now, point set is active')
 
def dist( a, b):
    return (a-b).mag();           
            
def mouse_point():
    return mPoint(py5.mouse_x-py5.width/2,py5.mouse_y-py5.height/2)

In [96]:
def random_adjacencia_matrix():
    matrix=np.random.randint(2,size=(10,10))
    for i in range(10):
        for j in range(10):
            if i==j:
                matrix[i][j]=0
            elif i<j:
                matrix[i][j]=matrix[j][i]
    return matrix

In [97]:
def setup():
    py5.size(1000, 1000)
    py5.rect_mode(py5.CENTER)
    global photo
    photo=py5.load_image("you win.jpeg")

num_of_points=7
points=np.random.rand(num_of_points,2)*1000-500
adjacencia_matrix=random_adjacencia_matrix()
ideig_ponthalmaz=[]
elek=[]
i=0

for x,y in points:
    random_csucshalmaz=set(j for j in range(10) if adjacencia_matrix[i][j]==1)
    ideig_ponthalmaz.append(mPoint(x,y,i,random_csucshalmaz))
    i+=1
graf_csucsai=dinamic_pointset(ideig_ponthalmaz)

ideiglenes_elhalmaz=[]
for i in range(num_of_points):
    for j in range(num_of_points):
        if adjacencia_matrix[i][j]==1:
            ideiglenes_elhalmaz.append(edge(graf_csucsai.plist[i],graf_csucsai.plist[j]))
            
graf_elei=edge_set(ideiglenes_elhalmaz)

def mouse_released(e):
    global graf_csucsai
    
    modifiers = e.get_modifiers()
    msgs = []
    if modifiers & e.SHIFT:
        graf_csucsai.add_point(mouse_point())
    else:
        graf_csucsai.do_something()

        

In [98]:
for point in graf_csucsai.plist:
    print(point)
    

(0,447.7532703524895,271.9661008028685,{8, 9, 3, 4})
(1,174.8417520669684,-70.46679472760644,{9, 3, 5})
(2,377.833714514485,-202.0876076225777,{5})
(3,-447.598580303037,385.53714373312596,{0, 1, 4, 5, 7, 9})
(4,-285.5085574228639,111.26369618180979,{0, 3, 5, 6, 7, 9})
(5,-108.8756486682658,1.616684250365779,{1, 2, 3, 4, 6})
(6,-22.20923526481147,-2.013852284363111,{8, 9, 4, 5})


In [99]:
len(graf_csucsai.plist)

7

In [100]:
    
def draw():
    # adatok
    py5.image(photo, 0, 0,100,100)
    global graf_csucsai
    mmouse_x=py5.mouse_x-py5.width/2
    mmouse_y=py5.mouse_y-py5.height/2
    graf_csucsai.update()
    if not(graf_elei.van_e_metszes()):
        py5.image(photo, 0, 0)
        
    
    #stílus
    py5.background(255)
    py5.translate(py5.width/2, py5.height/2);
    
    #rajzolás
    
    py5.stroke(0)
    py5.stroke_weight(10)
    graf_csucsai.display()
    graf_elei.display()
    
    points=graf_csucsai.plist
    n=len(graf_csucsai.plist)
    points_raw=[(p.x,p.y) for p in graf_csucsai.plist]
    
    
    
    
py5.run_sketch()
print("naon cep")

naon cep
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
F